In [1]:
import numpy as np
import math
import cvxpy as cp
from itertools import combinations
from itertools import permutations
import time
import copy
import pandas as pd
import copy

In [2]:
np.random.seed(1)

In [3]:
def collection_distribution(n,collection):
    #collection = list(combinations(range(n),size))
    collection_distribution = np.zeros((n,len(collection)))

    for i in range(len(collection)):
        distribution_S = np.random.exponential(1, len(collection[i]))
        assortment_dis = np.array([k/np.sum(distribution_S) for k in distribution_S])
        for j in range(len(collection[i])):
            collection_distribution[collection[i][j]][i]= assortment_dis[j]
            
    return collection_distribution

In [4]:
n = 7
instance_size = 100
collection_size = [2,3,5,10,15]
#collection_size = [2,3,5]
alpha = 1

''' instance_size = 1
collection_size = [2] '''

all_size2_ass = [list(x) for x in list(combinations(range(n),2))]
all_size3_ass = [list(x) for x in list(combinations(range(n),3))]
#all_size4_ass = [list(x) for x in list(combinations(range(n),4))]
#all_possible_assortment = all_size2_ass + all_size3_ass + all_size4_ass
all_possible_assortment = all_size2_ass + all_size3_ass 
print(len(all_possible_assortment))

std = 0.01

all_collection = [[] for _ in range(len(collection_size))]

# construct 1000 samples of random assortment size collection of first collection size
for j in range(instance_size):
    collection = []
    while(len(collection)<collection_size[0]):
        S = all_possible_assortment[np.random.randint(len(all_possible_assortment))]
        if S not in collection:
            collection.append(S)
    all_collection[0].append(collection)


for i in range(1,len(collection_size)):
    all_collection[i] = copy.deepcopy(all_collection[i-1])
    for j in range(instance_size):
        while(len(all_collection[i][j])<collection_size[i]):
            S = all_possible_assortment[np.random.randint(len(all_possible_assortment))]
            if S not in all_collection[i][j]:
                all_collection[i][j].append(S)

all_instance = []
for i in range(len(collection_size)):
    collection_ins = []   
    for j in range(instance_size):
        ''' v = np.random.normal(0,1,size=n) #standard normal
        curr_coll_mnl = collection_distribution_mnl(n,all_collection[i][j],v)
        collection_ins.append(gaussian_noise(curr_coll_mnl,std,alpha)) '''
        collection_ins.append(collection_distribution(n,all_collection[i][j]))
    all_instance.append(collection_ins)


56


In [5]:
def rum_limit(data):

    # data processing for RUM feasibility checking
    n,m = data.shape
    asmnt = []
    for i in range(m):
        curr_assortment = np.where(data[:,i]!=0)
        for j in curr_assortment:
            asmnt.append(list(j))
   
    # flatten the data to dimension 1
    data_zip = data.flatten()
    for j in np.where(data_zip!=0):
        index_in = j
    index_in.astype(int)
    index_in= list(index_in)

    #specify all the possible permutation lists for each element in the probability collection
    perm_tab = [[] for _ in range(n*m)]
    perm = list(permutations(range(n),n))
    perm = [list(i) for i in perm]

    for i in range(m):
        for j in asmnt[i]:
            for p in range(len(perm)):
            #for p in perm:
                new_p = [k for k in perm[p] if k in asmnt[i]]
                if j == new_p[0]:
                    perm_tab[i+j*m].append(p)
    
    lam = cp.Variable(shape=len(perm),nonneg = True)
    x = cp.Variable(shape = n*m ,nonneg = True)
    
    constraints = [sum(x[j] for j in asmnt[i])==1 for i in range(m)] # normalization conditions
    
    constraints = [sum(lam[i] for i in range(len(perm)))==1]
    for i in index_in:
        constraints+=[sum(lam[j] for j in perm_tab[i])- x[i] == 0]
    
    #obj = cp.Minimize(cp.norm1(x-data_zip))
    obj = cp.Minimize(sum(data_zip[k]*cp.abs(data_zip[k] - x[k]) for k in range(n*m)))
    
    prob = cp.Problem(obj,constraints)
    prob.solve(solver =cp.GUROBI)
    
    if prob.value < math.pow(10,-6):
        return [0,prob.solver_stats.solve_time] 
    else:
        return [prob.value,prob.solver_stats.solve_time] 

In [6]:
print('starting testing the limit of rum')
start_tinme = time.time()
rum_runtime = []
rum_total_loss = []  #of different collection size 
rum_mean_loss = []
rum_loss_se = []
rum_runtime_se = []
for i in range(len(all_instance)):
    loss = []
    runtime = [] # of the same collection size
    for j in range(len(all_instance[i])):
        result = rum_limit(all_instance[i][j])
        loss.append(result[0])
        runtime.append(result[1])
        
    test = pd.DataFrame({'loss':loss,'runtime':runtime})
    test.to_csv('rum/collection_'+str(collection_size[i])+'.csv')
    runtime = np.array(runtime)
    rum_runtime.append(np.mean(runtime))
    rum_runtime_se.append(np.std(runtime)/np.sqrt(len(runtime)))
    loss = np.array(loss)
    rum_mean_loss.append(np.mean(loss))
    rum_total_loss.append(np.sum(loss))
    rum_loss_se.append(np.std(loss)/np.sqrt(len(loss)))
    
print('the total running time',time.time()-start_tinme)

starting testing the limit of rum
Academic license - for non-commercial use only - expires 2023-07-14
Using license file /Users/autumn/gurobi.lic
the total running time 6691.868456125259


In [7]:
rum = pd.DataFrame({'collection_size':collection_size,'rum_mean_oss':rum_mean_loss,'rum_loss_se':rum_loss_se,'rum_mean_runtime':rum_runtime,'rum_runtime_se':rum_runtime_se})
rum.to_csv('rum_limit_summary.csv')

In [8]:
rum_mean_loss

[0.009100324947453378,
 0.04751833290086248,
 0.1370108106694161,
 0.56030484511451,
 1.1690020351278674]

In [9]:
data1 = np.array([[0.3,0.9,0],[0.7,0,0.8],[0,0.1,0.2]])
result1 = rum_limit(data1)
result1

[0, 0.0003337860107421875]

In [10]:
data2 = np.array([[7/20,2/8],[2/8,7/20],[2/5,0],[0,2/5]])
result2 = rum_limit(data2)
result2

[0, 0.0002968311309814453]

In [11]:
data3 = np.array([[0.1,0.2,0.2,0.25],[0.2,0.25,0.25,0.75],[0.2,0.55,0,0],[0.5,0,0.55,0]])
result3 = rum_limit(data3)
result3

[0.015000000000000038, 0.0004999637603759766]